In [1]:
import theano
from theano import tensor as T
from theano.tensor.nnet import conv

import numpy as np

In [2]:
rng = np.random.RandomState(23455)

# input (4D tensor)
input = T.tensor4(name='input')

# weights (4D tensor)
w_shape = (2, 3, 9, 9)
w_bound = np.sqrt(3 * 9 * 9)
W = theano.shared( np.asarray(
            rng.uniform(
                low=-1.0/w_bound, 
                high=1.0/w_bound, 
                size=w_shape),
            dtype=input.dtype), name ='W')

# biases (1D tensor)
b_shape = (2,)
b = theano.shared(np.asarray(
            rng.uniform(low=-.5, high=.5, size=b_shape),
            dtype=input.dtype), name ='b')

# convolution expression of input with filters W
# build symbolic expression to add bias and apply activation function, i.e. produce neural net layer output
Z = conv.conv2d(input, W) + b.dimshuffle('x', 0, 'x', 'x')

output = T.nnet.sigmoid(Z)

In [3]:
# create theano function to compute filtered images
f = theano.function([input], output)

In [ ]:
import pylab
from PIL import Image

In [ ]:
# open random image of dimensions 639x516
img = Image.open(open('images/3wolfmoon.jpg'))
# dimensions are (height, width, channel)
img = np.asarray(img, dtype='float64') / 256.

# put image in 4D tensor of shape (1, 3, height, width)
img_ = img.transpose(2, 0, 1).reshape(1, 3, 639, 516)
filtered_img = f(img_)

# plot original image and first and second components of output
pylab.subplot(1, 3, 1); pylab.axis('off'); pylab.imshow(img)
pylab.gray();
# recall that the convOp output (filtered image) is actually a "minibatch", of size 1 here, so we take index 0 in the first dimension:
pylab.subplot(1, 3, 2); pylab.axis('off'); pylab.imshow(filtered_img[0, 0, :, :])
pylab.subplot(1, 3, 3); pylab.axis('off'); pylab.imshow(filtered_img[0, 1, :, :])
pylab.show()